### Do the imports
- If I am doing any processing of files in a folder I like to import os and glob
- import cellpose models for training of inferance
- import something to read your images 
- import libraries to save your results


### Install pytorch if it isn't installed yeet

Use this website to configure the right installation packages:
[Pytorch](https://pytorch.org/get-started/locally/)

then use `pip install <list of packages>` to install it

For my macbook, it is

```pip3 install torch torchvision torchaudio```



In [1]:
import os
import glob
import sys

import tifffile # read an write tiff files
from cellpose import models
import napari
import torch
import nd2
import numpy as np

from skimage.transform import rescale

import utils

Create the napari viewer to visualize images and mask

In [2]:
viewer = napari.Viewer()

Find the image files to work with. Different path for Window, Mac, and Linux

In [30]:
if sys.platform == 'darwin': 
    imagedir = "/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data"
elif sys.plaform == 'win32':
    imagedir = os.path.join(['S:', 'micro',
                             'internal', 'cjw',
                             'PythonImageProcessing',
                             'cellpose',
                             'Data'])
else:
   imagedir = "/n/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data"
   
imagefiles = sorted(glob.glob(f"{imagedir}/*.tif"))

for i, f in enumerate(imagefiles):
    print(i, os.path.basename(f))

0 01-yeast_many.tif
1 02-a01_42.tif
2 03-a01_43.tif
3 04-nuclei_fish.tif
4 05-yeast_01.tif
5 06-yeast_02.tif
6 C-hela-cells.tif
7 fluo_00.tif


In [95]:
yeast = tifffile.imread(imagefiles[4])
yeast.shape

(512, 512)

Add the image to the napari viewer

In [96]:
viewer.layers.clear()
viewer.add_image(yeast)


<Image layer 'yeast' at 0x358df8580>

### Setup the cellpose model

#### GPU setting
- on windows or linux with an NVIDIA GPU, set `gpu=True``
- on M1 or M2 mac, set `device=torch.device('mps')`
- on old Mac or Windows without GPU, set `gpu=False` -- this will be slower

#### Model
- cellpose has a number of pretrained models to use. Start with `cyto` or `cyto2` (even it is nuclei)
- info about pretrained cellpose [models](https://cellpose.readthedocs.io/en/latest/models.html)

In [97]:
d = torch.device('mps')

# change gpu=True if on windows, and get rid of device
# model = models.Cellpose(gpu=True, model_type='cyto2')
model = models.Cellpose(gpu=False, device=d, model_type='cyto2')

### `model.eval`
Parameters
----------

The basic options:
- x : the image, can be a 2d numpy array, a list of numpy arrays,
or a 3d numpy array
- diameter : The approximate size of the object you are trying to segment
- channels :
    - [0, 0] for a grayscale image
    - [] 

Returns
-------
- masks : An array or list of arrays with segmenation labels/masks
- flows : A list of numpy arrays with fow

#### Try cellpose with no paramter changes

In [98]:

masks, flows, styles, diams = model.eval(yeast, channels=[0, 0])

Show the results from cellpose a layers in napari

In [99]:
viewer.layers.clear()
viewer.add_image(yeast)
viewer.add_labels(masks)


<Labels layer 'masks' at 0x2f3085330>

In [ ]:
model.eval?

See what happens when the diameter is changed. What diameter is the best?

In [101]:
## start at 50, then adjust
masks, flows, styles, diams = model.eval(yeast, channels=[0, 0],
                                         diameter=45)

In [102]:
viewer.layers.clear()
viewer.add_image(yeast)
viewer.add_labels(masks)

<Labels layer 'masks' at 0x35d861b40>

In [89]:
viewer.add_image(flows[0], visible=False, name="xyflow")
viewer.add_image(flows[1][0], visible=False, name="xflow")
viewer.add_image(flows[2], visible=False, name="probability")

<Image layer 'probability' at 0x2f3086650>

In [111]:
y2 = rescale(yeast, 30/45, preserve_range=True, anti_aliasing=True, anti_aliasing_sigma=.5)

In [112]:

masks, flows, styles, diams = model.eval(y2, channels=[0, 0])

In [113]:
viewer.layers.clear()
viewer.add_image(y2)
viewer.add_labels(masks)

<Labels layer 'masks' at 0x361c274f0>

In [114]:
viewer.add_image(flows[0], visible=False, name="xyflow")
viewer.add_image(flows[2], visible=False, name="probability")

<Image layer 'probability' at 0x35df5f6a0>